In [1]:
import networkx as nx
import numpy as np
from numpy.random import normal
from collections import defaultdict
import matplotlib.pyplot as plt
import os
from tqdm import tqdm 

In [2]:
class Product:
    
    def __init__(self, product_name="a", mean=0.5, std=0.167): 
        self.name = product_name
        self.quality = normal(mean, std) 

In [3]:
class Consumer:
    
    def __init__(self, consumer_id, products):

        self.products = products
        
        #personal attributes
        self.consumer_id = consumer_id 
        self.follower_tendency = normal(0.5, 0.167) 
        
        #product attributes 
        self.quality_perceived = {product.name : product.quality for product in self.products}
        self.minimum_satisfaction_quality = {product.name : 0.5 for product in self.products} #default
        self.purchased = {product.name : False for product in self.products}
        
        self.utility = {product.name : 0.0 for product in self.products} 
        self.purchase_threshold = 0.5
        
    
    def update_consumer_belief(self, product, neighbours):
        
        Q_i   = self.quality_perceived[product]
        Q_des = self.minimum_satisfaction_quality[product]
        product_satisfaction = Q_i - Q_des
        
        N_i = 0
        N_p = 0
                
        for neighbour in neighbours:
            if neighbour.purchased[product]==True:
                N_i+=1
            
            for prod in self.products: 
                if neighbour.purchased[prod.name]==True:
                    N_p+=1
                    
        utility = (1 - self.follower_tendency) * product_satisfaction + self.follower_tendency * (N_i/(N_p+1))
    
        self.utility[product] = utility 
        
        
        ''' Purchase product only if it utility is above some threshold '''
        if self.purchased[product]!=True and utility >= self.purchase_threshold: 
            self.purchased[product] = True
    


In [4]:
class Social_Network:
    
    def __init__(self, number_of_consumers, products, market_share):
        
        self.products = products
        
        self.number_of_consumers = number_of_consumers
        self.connectedness_k =  10
        self.consumer_edge_probability = 0.2
        
        self.market_share = market_share 
        
        self.seed = 123
        
        self.make_social_network()
    
    def make_social_network(self): 
        
        self.social_network = nx.watts_strogatz_graph(self.number_of_consumers, 
                                                      k = self.connectedness_k,
                                                      p = self.consumer_edge_probability, 
                                                      seed = self.seed)  
        
        
        self.neighbours = defaultdict(list)
        for u, v in self.social_network.edges:
            self.neighbours[u].append(v)
            
        self.consumer_directory = defaultdict()
        
        for consumer_id in self.social_network.nodes:
            
            consumer = Consumer(consumer_id, self.products)            
            self.consumer_directory[consumer_id] = consumer
            
        for consumer_id in self.social_network.nodes:
            
            neighbours = self.neighbours[consumer_id]
            self.neighbours[consumer_id] = [self.consumer_directory[n] for n in neighbours]
            
            product = np.random.choice(self.products, 1, p = self.market_share)
            self.consumer_directory[consumer_id].purchased[product[0].name] = True
        
            
    def update_social_network(self): 
        
        for consumer_id, consumer in self.consumer_directory.items():
            for product in self.products: 
                consumer.update_consumer_belief(product.name, self.neighbours[consumer_id])
        
    def get_summary(self):
        
        utility = defaultdict(list)
        purchases = defaultdict(list)
        for consumer_id, consumer in self.consumer_directory.items():
            for product in self.products: 
                utility[product.name].append(consumer.utility[product.name])
                purchases[product.name].append(1 if consumer.purchased[product.name] else 0)
         
        print()
        for product in self.products: 
            print("Product: {}, Utility: {}, Purchases: {}".format(product.name, np.mean(utility[product.name]), np.sum(purchases[product.name])))
        print()
        
    def save_social_network(self, timestep=0): 
        
        plt.figure(figsize =(10, 7))
  
        color = {'Canva': 'blue',
                 'CCX': 'red'}
    
        node_color = ['red' if self.consumer_directory[consumer_id].purchased['CCX'] else 'blue' for consumer_id in self.social_network.nodes]

        nx.draw_circular(self.social_network,  node_color = node_color, node_size=3)

        plt.title('Simulation Time Step t-{}'.format(str(timestep)))
        #plt.tight_layout();
                
        try:
            os.mkdir('simulations')
        except:
            pass
            #print("Folder already exists")
            
        plt.savefig('simulations/t_{}.png'.format(timestep))
        
        
        

In [5]:
def simulate(number_of_consumers=30, num_of_iterations=5, market_share=None, products=None): 
    
    for iteration in tqdm(range(num_of_iterations)):
        
        if iteration==0: 
            social_network = Social_Network(number_of_consumers, 
                                            products = products, 
                                            market_share = market_share
                                            )
            #social_network.save_social_network(iteration) 
            social_network.get_summary()
        
        else:
            social_network.update_social_network()
            
            if iteration==num_of_iterations-1: 
                #social_network.save_social_network(iteration)
                social_network.get_summary()
            
    return social_network
        

In [35]:
#When both have same Quality distributions and Market Share
products = [Product(product_name='CCX'), Product(product_name='Canva')] #Assuming same quality distribution 

social_network = simulate(number_of_consumers=100000, 
                          num_of_iterations=200, 
                          market_share = [0.5, 0.5], 
                          products = products)

  0%|          | 1/200 [00:05<19:31,  5.89s/it]


Product: CCX, Utility: 0.0, Purchases: 49935
Product: Canva, Utility: 0.0, Purchases: 50065



100%|██████████| 200/200 [02:02<00:00,  1.64it/s]


Product: CCX, Utility: 0.30905090956611464, Purchases: 52434
Product: Canva, Utility: 0.20566988265546618, Purchases: 51007



In [9]:
#When both have different Quality distributions and same Market Share
products = [Product('CCX', 0.5, 0.167), Product('Canva', 0.7, 0.1)] 

social_network = simulate(number_of_consumers=100000, 
                          num_of_iterations=200, 
                          market_share = [0.5, 0.5], 
                          products = products)

  0%|          | 1/200 [00:05<19:46,  5.96s/it]


Product: CCX, Utility: 0.0, Purchases: 50193
Product: Canva, Utility: 0.0, Purchases: 49807



100%|██████████| 200/200 [01:55<00:00,  1.74it/s]


Product: CCX, Utility: 0.27422158635690186, Purchases: 51979
Product: Canva, Utility: 0.27731930770978275, Purchases: 51577



In [6]:
#When both have same Quality distributions and different Market Share
products = [Product(product_name='CCX'), Product(product_name='Canva')] #Assuming same quality distribution 

social_network = simulate(number_of_consumers=100000, 
                          num_of_iterations=200, 
                          market_share = [0.30, 0.70], 
                          products = products)

  0%|          | 1/200 [00:05<18:52,  5.69s/it]


Product: CCX, Utility: 0.0, Purchases: 29908
Product: Canva, Utility: 0.0, Purchases: 70092



100%|██████████| 200/200 [01:59<00:00,  1.67it/s]


Product: CCX, Utility: 0.1649592980504369, Purchases: 30212
Product: Canva, Utility: 0.42166608728276334, Purchases: 76632



In [8]:
#When both have different Quality distributions and different Market Share
products = [Product('CCX', 0.5, 0.167), Product('Canva', 0.7, 0.1)] 

social_network = simulate(number_of_consumers=100000, 
                          num_of_iterations=200, 
                          market_share = [0.30, 0.70], 
                          products = products)

  0%|          | 1/200 [00:05<18:23,  5.55s/it]


Product: CCX, Utility: 0.0, Purchases: 30016
Product: Canva, Utility: 0.0, Purchases: 69984



100%|██████████| 200/200 [01:56<00:00,  1.71it/s]


Product: CCX, Utility: 0.17122121413096883, Purchases: 30332
Product: Canva, Utility: 0.3661432075249108, Purchases: 74020

